In [5]:
import pandas as pd, numpy as np, scipy.stats as stats, matplotlib.pyplot as plt, seaborn as sns, sklearn.preprocessing as prep
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, StandardScaler, OneHotEncoder, RobustScaler, LabelEncoder
from sklearn.feature_selection import SelectKBest, chi2, f_classif
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from scipy.stats.mstats import winsorize
from datetime import datetime

### copy z prvej fazy zadania

In [6]:
df_product = pd.read_csv("product.csv", sep='\t')
df_session = pd.read_csv("session.csv", sep='\t')
df_users = pd.read_csv("user.csv", sep='\t')

print(len(df_product), len(df_session), len(df_users))

with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    print(df_session)

14908 11039 2592


IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [7]:
df_nodupuser = df_users
idecka = {}
for id in df_users["user_id"]:
    if id in idecka.keys():
        idecka[id] += 1
    else:
        idecka[id] = 1
to_remove = []
for row in df_users.iterrows():
    index = row[0]
    user_id = row[1]["user_id"]
    if idecka[user_id] > 1:
        to_remove.append(index)
print(to_remove)
df_nodupuser = df_nodupuser.drop(to_remove)
print(len(df_users), len(df_nodupuser))
df_nodupuser.reset_index()

[0, 3, 4, 5, 6, 7, 8, 10, 13, 14, 15, 16, 17, 20, 21, 22, 25, 26, 28, 31, 32, 33, 37, 38, 39, 41, 43, 44, 45, 46, 47, 49, 50, 51, 53, 56, 57, 58, 59, 62, 64, 65, 66, 67, 68, 69, 71, 74, 75, 77, 78, 79, 80, 81, 82, 83, 84, 86, 92, 94, 97, 100, 101, 102, 107, 109, 111, 114, 115, 116, 117, 118, 121, 122, 123, 124, 125, 126, 127, 128, 129, 131, 132, 133, 134, 136, 138, 139, 140, 141, 144, 145, 148, 149, 151, 152, 153, 154, 155, 156, 160, 162, 163, 164, 165, 166, 168, 169, 174, 175, 176, 177, 182, 184, 186, 187, 190, 191, 192, 194, 196, 198, 202, 205, 206, 207, 211, 212, 213, 214, 215, 216, 218, 219, 220, 221, 223, 224, 225, 226, 227, 229, 230, 231, 232, 233, 235, 236, 237, 240, 241, 242, 243, 244, 247, 248, 249, 250, 252, 253, 259, 260, 261, 262, 263, 265, 266, 267, 268, 271, 272, 273, 274, 276, 277, 278, 281, 282, 283, 284, 285, 286, 287, 288, 292, 293, 296, 299, 300, 301, 302, 303, 305, 306, 307, 308, 309, 310, 311, 313, 315, 316, 318, 320, 321, 322, 323, 325, 326, 328, 329, 330, 332, 33

,index,race,username,sex,residence,user_id,name,address,mail,current_location,registration,birthdate
0,1,unspecified,pirescarolina,F,NaN,2066,Pietra Castro,"Campo de Araújo, 606\nMonte São José\n57558095...",gomesana@yahoo.com.br,"(Decimal('28.1621345'), Decimal('-173.517410'))",2021-08-09,NaN
1,2,NaN,maskovalucie,NaN,NaN,1023,Růžena Pospíšilová,Kremnická 66\n518 24 Žacléř,pavelfiala@post.cz,"(Decimal('-80.913702'), Decimal('21.046467'))",2020/08/06,1921-09-21
2,9,NaN,gda-cruz,NaN,NaN,2061,Emanuel da Conceição,NaN,gmonteiro@yahoo.com.br,"(Decimal('89.734118'), Decimal('-103.039006'))","11/07/2018, 00:00:00",2000-11-28
3,11,NaN,trommlerleonie,M,NaN,96,Leo Hiller-Holt,Täschering 491\n69752 Eberswalde,daniellesontag@gmx.de,"(Decimal('-81.5533295'), Decimal('-72.476163'))",12 Jan 2014,1937-11-17
4,12,NaN,igorcostela,NaN,NaN,1196,Beatriz Carvalho,"Loteamento Nathan Melo, 8\nSantana Do Cafezal\...",fogacaluiz-otavio@bol.com.br,"(Decimal('80.7165165'), Decimal('-17.568838'))",25 Nov 2013,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
944,2575,NaN,smithcarmen,M,"564 Andrea Freeway Suite 701\nWilliamsmouth, N...",1201,Keith Patton DDS,04212 Melvin Mountain Suite 359\nSouth Douglas...,elizabeth32@hotmail.com,"(Decimal('-70.997122'), Decimal('156.990661'))",2021-01-12,1975-01-27
945,2578,NaN,wbarber,M,NaN,1551,Eduardo Johnson,"408 Davis Junction Apt. 020\nMejiaside, PA 62823",hvilla@hotmail.com,"(Decimal('35.572745'), Decimal('13.032466'))","03/31/2018, 00:00:00",1951-11-07
946,2581,NaN,xwatanabe,F,NaN,2093,松本 香織,岩手県山武郡九十九里町虎ノ門虎ノ門ヒルズ森タワー4丁目24番12号 ハイツ港南664,yamamotorika@yahoo.com,"(Decimal('-12.9191985'), Decimal('-21.733244'))",14 Oct 2013,2019-02-17
947,2583,white,nogueiragabriel,NaN,NaN,2511,Arthur Cardoso,NaN,correiaalana@bol.com.br,"(Decimal('-39.812127'), Decimal('0.187696'))","07/05/2022, 00:00:00",1968-08-08


In [8]:
df_merged = df_session.merge(df_nodupuser, how="inner")
df_merged

,session_id,session_duration,pct_mouse_click,pct_mouse_move,pct_scrandom,browser_name,pct_scroll_move,wild_mouse_duration,total_load_time,screen_width,...,race,username,sex,residence,name,address,mail,current_location,registration,birthdate
0,223045,47.96486,11.32534,11.29606,48.47653,firefox,48.95236,11.39144,2.30071,1280,...,black,gscalfaro,NaN,NaN,Gianni Sagese,"Piazza Silvio, 97 Appartamento 98\n67058, San ...",arturocostalonga@libero.it,"(Decimal('-72.364280'), Decimal('101.224035'))",2021-09-11,1938-10-26
1,623734,132.25714,11.88174,11.58618,25.58663,safari,51.07750,9.44179,1.84508,lower,...,black,gscalfaro,NaN,NaN,Gianni Sagese,"Piazza Silvio, 97 Appartamento 98\n67058, San ...",arturocostalonga@libero.it,"(Decimal('-72.364280'), Decimal('101.224035'))",2021-09-11,1938-10-26
2,159057,143.60687,7.83379,2.92107,70.63719,opera,41.89119,10.05044,2.92902,lower,...,black,gscalfaro,NaN,NaN,Gianni Sagese,"Piazza Silvio, 97 Appartamento 98\n67058, San ...",arturocostalonga@libero.it,"(Decimal('-72.364280'), Decimal('101.224035'))",2021-09-11,1938-10-26
3,551040,248.09308,4.95972,4.52411,48.16995,opera,54.76994,9.96582,8.07877,1366,...,black,gscalfaro,NaN,NaN,Gianni Sagese,"Piazza Silvio, 97 Appartamento 98\n67058, San ...",arturocostalonga@libero.it,"(Decimal('-72.364280'), Decimal('101.224035'))",2021-09-11,1938-10-26
4,6355,165.90198,11.27945,14.79490,49.78113,chrome,44.72991,8.48023,2.04936,800,...,black,gscalfaro,NaN,NaN,Gianni Sagese,"Piazza Silvio, 97 Appartamento 98\n67058, San ...",arturocostalonga@libero.it,"(Decimal('-72.364280'), Decimal('101.224035'))",2021-09-11,1938-10-26
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4070,817978,168.30985,13.22728,13.23870,59.44815,edge,43.64091,11.70369,1.05211,1280,...,asian,ccerna,M,Pyšelská 1\n369 25 Žamberk,Miloslav Dušek,NaN,krejcovabohumila@centrum.cz,"(Decimal('39.6622325'), Decimal('-62.817658'))","12/10/2021, 00:00:00",1964-12-12
4071,992036,172.54963,11.39298,14.91608,41.20110,edge,47.24597,9.50976,2.63677,higher,...,NaN,agnolosanguineti,NaN,NaN,Sig. Micheletto Dalla,"Contrada Roman, 975 Appartamento 5\n00127, Mez...",paloma99@gmail.com,"(Decimal('73.519198'), Decimal('-116.041196'))","03/16/2017, 00:00:00",1965-11-23
4072,856525,92.97564,9.92790,7.32274,59.13477,chrome,58.16094,10.43395,5.45315,800,...,NaN,milena14,F,NaN,Bianca Fogaça,NaN,sarahcardoso@ig.com.br,"(Decimal('89.4235045'), Decimal('-41.293519'))",2018-06-20,2006-09-19
4073,472623,54.60196,10.88118,11.77482,44.68072,chrome,57.48251,7.80662,4.27341,1280,...,NaN,milena14,F,NaN,Bianca Fogaça,NaN,sarahcardoso@ig.com.br,"(Decimal('89.4235045'), Decimal('-41.293519'))",2018-06-20,2006-09-19


In [9]:
# rrrr-mm-dd
# rrrr/mm/dd
# mm/dd/rrrr, 00:00:00
# dd Mes rrrr

def get_format(date):
    months = {
        "Jan": 1,
        "Feb": 2,
        "Mar": 3,
        "Apr": 4,
        "May": 5,
        "Jun": 6,
        "Jul": 7,
        "Aug": 8,
        "Sep": 9,
        "Oct": 10,
        "Nov": 11,
        "Dec": 12
    }
    if len(date.split("-")) == 3:
        splitted = date.split("-")
        rok = splitted[0]
        mesiac = splitted[1]
        den = splitted[2]
    elif len(date.split("/")) == 3:
        splitted = date.split("/")
        if len(splitted[2]) > 4:
            mesiac = splitted[0]
            den = splitted[1]
            rok = splitted[2][:4]
        else:
            rok = splitted[0]
            mesiac = splitted[1]
            den = splitted[2]
    elif len(date.split(" ")) == 3:
        splitted = date.split(" ")
        den = splitted[0]
        mesiac = months[splitted[1]]
        rok = splitted[2]
    return f"{rok}-{mesiac}-{den}"


for row in df_merged.iterrows():
    datum = get_format(row[1]["registration"])
    df_merged.at[row[0], "registration"] = datum


# 2. faza

### Data transformation
- premena session_start na sekudy od epochu_startu pre numerickost dat
- a zmena registracie na ciselnu hodnotu zmenou z 2018-08-12 na 20180812 co znamena ze vacsi datum je aj numericky vacsi

In [10]:
from datetime import datetime


def to_seconds_since_epoch(date_str):
    date_format = "%Y-%m-%d %H:%M:%S"  # Adjust this format to match the format in your DataFrame
    date = datetime.strptime(date_str, date_format)
    epoch_start = datetime(1970, 1, 1)
    return (date - epoch_start).total_seconds()


def to_int_from_registration(regtistration):
    return int(''.join(regtistration.split("-")))


### Vymazavanie dat podla pomeru NaN hodnot
- ak je pomer NaN vacsi ako zvoleny percentage stlpec sa vymaze
- taktiez sa vymazavaju stlpce s identifikacnymi hodnotami ako napr session_id, user_id, product_ean
- zaroven sa vymazavaju string stlpce, ktore maju vysoku diverzitu a zaroven nemaju vela spolocneho s predikovanim vysledku (bohuzial aj current location kedze je nahodne vygenerovana)

In [11]:

def NaN_pomer_del(df, percentage):
    ratio = int(((100 - percentage) / 100) * df.shape[0] + 1)
    nan_percentage = df.isna().mean() * 100
    print("deleted rows:\n", nan_percentage[nan_percentage > 30])
    df = df.dropna(axis=1, thresh=ratio)
    df = df.drop(
        columns=['session_id', 'user_id', 'product_ean', 'name', 'username', 'address', 'mail', 'current_location'],
        axis=1)
    return df

# Integrácia a čistenie dát
#### odstránenie pozorovaní s chýbajúcimi údajmi

#### nahradenie chýbajúcej hodnoty priemerom


In [12]:
def remove_missing_rows(df):
    df = df_merged.dropna()
    return df


def replace_missing_All(df):
    """
    :param df: dataframe
    :return: dataframe with replaced empty values of numerical columns with average of that column
    """
    for column in df.columns:

        if pd.api.types.is_numeric_dtype(df[column]):

            mean_value = df[column].mean()
            df[column].fillna(mean_value, inplace=True)
        elif not pd.api.types.is_numeric_dtype(df[column]):
            df[column].fillna('unspecified', inplace=True)
    return df

# Integrácia a čistenie dát
#### odstránenie vychýlených (odľahlých) pozorovaní
- odstrániť údaje je vhodné napríklad, ak je session_duration príliš veľké alebo malé.
#### nahradenie vychýlenej hodnoty hraničnými hodnotami rozdelenia (napr. 5%, 95%)
- nahradenie outlierov meanom daného stĺpca

In [13]:

def remove_outliers_for_num_column(df, bottom, top, all=False, column_names=None):
    """
    :param df: dataframe
    :param column_names: list/tuple of columns to process
    :param bottom: lower quantile bound, 0-1
    :param top: upper quantile bound, 0-1
    :param all: True if this should be done for all columns
    :return: dataframe without outliers
    """
    if all:
        for column in df.select_dtypes(include=['number']):
            lower_percentile = df[column].quantile(bottom)
            upper_percentile = df[column].quantile(top)

            df_cleaned = df[(df[column] >= lower_percentile) & (df[column] <= upper_percentile)].dropna()
    else:
        for column in df:
            if not pd.api.types.is_numeric_dtype(df[column]):
                continue
            lower_percentile = df[column].quantile(bottom)
            upper_percentile = df[column].quantile(top)

            df_cleaned = df[(df[column] >= lower_percentile) & (df[column] <= upper_percentile)].dropna()
    return df_cleaned


def replace_outliers_for_num_column(df, bottom, top, all=False, column_names=None):
    """
    :param df: dataframe
    :param column_names: list/tuple of columns to process
    :param bottom: lower quantile bound, 0-1
    :param top: upper quantile bound, 0-1
    :param all: True if this should be done for all columns
    :return: dataframe with replaced outliers
    """
    if all:
        for column in df.select_dtypes(include='number').columns:
            lower_threshold = df[column].quantile(bottom)
            upper_threshold = df[column].quantile(top)

            df[column] = df[column].apply(
                lambda x: lower_threshold if x < lower_threshold else (upper_threshold if x > upper_threshold else x))

    else:
        for column in df:

            if not pd.api.types.is_numeric_dtype(df[column]):
                continue
            lower_threshold = df[column].quantile(bottom)
            upper_threshold = df[column].quantile(top)

            df[column] = df[column].apply(
                lambda x: lower_threshold if x < lower_threshold else (upper_threshold if x > upper_threshold else x))

    return df


## Realizacia predspracovania dat

### Transformers a Scalers
- po testovani 5 transformerov nam vysli boxcox a pow_one_point_two ako 2 najlepsie transformery a tak vyuzivame tie
- pre scalery pouzivame na porovnanie MinMaxScaler, StandardScaler a RobustScaler (implementovany az pri pipeline)
    - funkcie nizsie su tu len pre rozdelenie a strukturovanie odovzdania kedze su nasledne prepisane aby fungovali ako class pre pipeline

In [14]:

def qq_compare(df, column):
    plt.subplot(1, 2, 1)
    stats.probplot(df[column], dist='norm', plot=plt)
    plt.subplot(1, 2, 2)
    sns.histplot(df[column])
    plt.show()


def box_cox_transf(df, column):
    boxcox_t = pd.DataFrame(df[column])
    boxcox_t[column], param = stats.boxcox(boxcox_t[column])

    return [boxcox_t, param]


def pow_pointtwo_t(df, column):
    pow_pointtwo_t = pd.DataFrame((df[column]) ** (1 / 1.2))
    return pow_pointtwo_t


def standard_scaler(type, df):
    stand_scaler = StandardScaler()
    if type == 'train':
        df_scaled = stand_scaler.fit_transform(df)
    else:
        df_scaled = stand_scaler.transform(df)
    return df_scaled


def min_max_scaler(type, df):
    min_max = MinMaxScaler()
    if type == 'train':
        df_scaled = min_max.fit_transform(df)
    else:
        df_scaled = min_max.transform(df)
    return df_scaled



### 3 sposoby feature selection
- SelectKBest so skorovaciou funckiou f_classif - pouzita pretoze je dobra na klasifikovanie numerickych atributov co je vacsina nasich atributov
- Select podla korelacie s predikovanou premennou na porovnanie
- SelectKBest so skorovaciou funkciou chi-squared - pouzita len na porovnanie kedze chi squared vyzaduje pozitivne cisla, moze sa teda pouzit len zaroven s MinMaxScalerom pretoze ostatne scalery vytvoria aj negativne hodnoty

In [15]:
def K_best_classif(df, ack_df, n):
    best_feat = SelectKBest(score_func=f_classif, k=n)
    fit = best_feat.fit(df, ack_df)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(df.columns)
    feature_scores = pd.concat([dfcolumns, dfscores], axis=1)
    feature_scores.columns = ["feature", "score"]
    print(feature_scores.sort_values('score', ascending=False).nlargest(10, 'score'))


def K_best_chi2(df, ack_df, n):
    best_feat = SelectKBest(score_func=chi2, k=n)
    fit = best_feat.fit(df, ack_df)
    dfscores = pd.DataFrame(fit.scores_)
    dfcolumns = pd.DataFrame(df.columns)
    feature_scores = pd.concat([dfcolumns, dfscores], axis=1)
    feature_scores.columns = ["feature", "score"]
    print(feature_scores.sort_values('score', ascending=False).nlargest(10, 'score'))


def corr_best(df, k):
    corr_max = abs(df.corr())
    print(corr_max['ack'].nlargest(k))




### Vytvorenie pipeline
- vyuzity sklearn.pipeline
- kazda trieda predstavuje jeden krok pipeline
- zostavovanie pipeline je modularne co znamena ze sa napr. rozne transformery sa mozu  zamienat iba zmenou nazvu triedy v pipeline

In [16]:


def split_train_test(df):
    predictor = df['ack'].copy()
    independants = df.drop('ack', axis=1).copy()
    independants_train, independants_test, predictor_train, predictor_test = train_test_split(independants, predictor,
                                                                                              test_size=0.2,
                                                                                              random_state=42)
    predictor_train = predictor_train.reset_index().drop('index', axis=1)
    predictor_test = predictor_test.reset_index().drop('index', axis=1)
    independants_train = independants_train.reset_index().drop('index', axis=1)
    independants_test = independants_test.reset_index().drop('index', axis=1)
    return [predictor_train, predictor_test, independants_train, independants_test]


mnoziny = split_train_test(df_merged)
predictor_train, predictor_test, independants_train, independants_test = mnoziny[0], mnoziny[1], mnoziny[2], mnoziny[3]


class ToIntTransformations(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['session_start'] = X['session_start'].apply(to_seconds_since_epoch)
        X['registration'] = X['registration'].apply(to_int_from_registration)
        return X


class HighNanDropPlusUselessColDeletion(BaseEstimator, TransformerMixin):
    def __init__(self, percentage):
        self.percentage = percentage

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        ratio = int(((100 - self.percentage) / 100) * X.shape[0] + 1)
        nan_percentage = X.isna().mean() * 100
        print("deleted rows:\n", nan_percentage[nan_percentage > 30])
        X = X.dropna(axis=1, thresh=ratio)
        X = X.drop(
            columns=['session_id', 'user_id', 'product_ean', 'name', 'username', 'address', 'mail', 'current_location'],
            axis=1)
        return X


class NanRemover(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = X.dropna()
        return X


class NanReplacerBoth(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = replace_missing_All(X)
        return X


class RemoveOutliers(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = remove_outliers_for_num_column(X, 0.02, 0.98)
        return X


class ReplaceOutliers(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X = replace_outliers_for_num_column(X, 0.02, 0.98)
        return X


class BoxCoxTransformer(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.lambdas_ = None

    def fit(self, X, y=None):
        self.lambdas_ = []
        for column in X:
            _, lambda_ = stats.boxcox(X[column] + abs(X[column].min()) + 1)
            self.lambdas_.append(lambda_)
        return self

    def transform(self, X):
        transformed_X = X.copy()
        for column, lambda_ in enumerate(self.lambdas_):
            transformed_X.iloc[:, column] = stats.boxcox(X.iloc[:, column] + abs(X.iloc[:, column].min()) + 1,
                                                         lmbda=lambda_)

        return transformed_X


class PowPointTwoTransformer(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        transforemed_X = X.copy()
        for column in transforemed_X:
            transforemed_X[column] = pd.DataFrame((X[column]) ** (1 / 1.2))
        return transforemed_X


class StandardScalerMine(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = StandardScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        holder = X.copy()
        X = self.scaler.transform(X)
        X = pd.DataFrame(X, columns=holder.columns)
        return X


class MinMaxScalerMine(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = MinMaxScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        holder = X.copy()
        X = self.scaler.transform(X)
        X = pd.DataFrame(X, columns=holder.columns)
        return X


class RobusScalerMine(BaseEstimator, TransformerMixin):
    def __init__(self):
        self.scaler = RobustScaler()

    def fit(self, X, y=None):
        self.scaler.fit(X)
        return self

    def transform(self, X):
        holder = X.copy()
        X = self.scaler.transform(X)
        X = pd.DataFrame(X, columns=holder.columns)
        return X


class OneHotEncoderMine(BaseEstimator, TransformerMixin):
    #enkodovanie kategorickych dat pomocou OneHotEncoder ktory rozdeli kazdu kategoriu stlpca na vlastny stlpec, 
    #ktory bude mat hodntou 1 tam kde bola hodnota v originalnom stlpci a 0 na zvysku
    def fit(self, X, y=None):

        return self

    def transform(self, X):
        result = pd.DataFrame(columns=['a'])
        x = 0
        for column in X:
            encoder = OneHotEncoder()
            transform = encoder.fit_transform(X[[column]]).toarray()
            categories = encoder.categories_
            categories = np.array(categories).ravel()
            if x == 1:
                categories += '_W'
            elif x == 2:
                categories += '_H'
            temp_df = pd.DataFrame(transform, columns=categories)
            result = pd.concat([result, temp_df], axis=1)
            x += 1
        return result.drop('a', axis=1)


#touto pipline si prechadzaju vsetky data su v nej zakladne transformacie na zlepsie formatu dataframu
pipeline_pre_encoding_transforming = Pipeline([
    ('highdropper', HighNanDropPlusUselessColDeletion(30)),
    ('tointtrans', ToIntTransformations()),
    ('NanSolution', NanReplacerBoth())
])

#pipeline pre numericke data
pipeline_num_transforming = Pipeline([
    ('outlierhandling', ReplaceOutliers()),
    ('transformer', PowPointTwoTransformer()),
    ('scaler', RobusScalerMine())
])

# pipeline pre kategoricke data
pipeline_categ_transforming = Pipeline([
    ('encoder', OneHotEncoderMine())
])

#inicialna pipeline len na pripravenie dat na transformovanie
pre_trans_encode_All = pipeline_pre_encoding_transforming.fit_transform(df_merged)

# rozdelenie na trian a test susety
mnoziny = split_train_test(pre_trans_encode_All)
predictor_train, predictor_test, independants_train, independants_test = mnoziny[0], mnoziny[1], mnoziny[2], mnoziny[3]


# column_transformer pre paralelne spracovanie
# column_transformer = ColumnTransformer([
#     ('numerical', pipeline_num_transforming, independants_train.select_dtypes(include='number').columns),
#     ('categorical', pipeline_categ_transforming, independants_train.select_dtypes(exclude='number').columns)
# ])

# # vysledok nie je vo forme data frame preto bola vytvorena funkcia scaling
# col_trans = column_transformer.fit_transform(independants_train)


def scaling_train(df, ack_df):
    nums_scaled_transformed = pipeline_num_transforming.fit_transform(df.select_dtypes(include='number'))
    cats_scaled_tranformed = pipeline_categ_transforming.fit_transform(df.select_dtypes(exclude='number'))
    # pouziva sa dropna pri returne pretoze pri pouziti remove ourliers na numerickych datach sa nasledne pri concate vytvorili NaN kedze remove outliers nebol pouzity na kategoricke data
    # toto nie je problem pri rieseni outlierov pomocou nahradzovania
    return pd.concat([nums_scaled_transformed, cats_scaled_tranformed, ack_df], axis=1).dropna()


def scaling_test(df, ack_df):
    nums_scaled_transformed = pipeline_num_transforming.transform(df.select_dtypes(include='number'))
    cats_scaled_tranformed = pipeline_categ_transforming.transform(df.select_dtypes(exclude='number'))
    # pouziva sa dropna pri returne pretoze pri pouziti remove ourliers na numerickych datach sa nasledne pri concate vytvorili NaN kedze remove outliers nebol pouzity na kategoricke data
    # toto nie je problem pri rieseni outlierov pomocou nahradzovania
    return pd.concat([nums_scaled_transformed, cats_scaled_tranformed, ack_df], axis=1).dropna()


final_frame_train = scaling_train(independants_train, predictor_train)
final_frame_test = scaling_test(independants_test, predictor_test)

K_best_classif(final_frame_train.drop('ack', axis=1), final_frame_train['ack'], 10)
print('-------------------')
corr_best(final_frame_train, 10)

K_best_classif(final_frame_test.drop('ack', axis=1), final_frame_test['ack'], 10)
print('-------------------')
corr_best(final_frame_test, 10)


deleted rows:
 race         70.552147
sex          49.055215
residence    63.411043
birthdate    46.061350
dtype: float64
                   feature       score
1          pct_mouse_click  602.705958
12         pct_doubleclick  596.707260
15  page_activity_duration  542.650179
2           pct_mouse_move  188.909235
29                  1920_W    6.988940
18            registration    5.068868
9   pct_click_product_info    3.913190
4          pct_scroll_move    3.553247
26                  1024_W    2.810677
0         session_duration    2.541455
-------------------
ack                       1.000000
pct_mouse_click           0.395111
pct_doubleclick           0.393446
page_activity_duration    0.377860
pct_mouse_move            0.234106
1920_W                    0.046266
registration              0.039413
pct_click_product_info    0.034636
pct_scroll_move           0.033007
1024_W                    0.029359
Name: ack, dtype: float64
                     feature       score
12          

### Vysledok feature selection
- vysledky su zoradene podla atributov s najlepsim skore cize nalepsi potencial na predikovanie ack
- z vysledku vieme nasledne vybrat x atributov na nasledne trenovanie ML modelu
- pocet vybranych atributov sa moze lubovolne menit

# 50 / 50

In [42]:
temp_df = independants_train
temp_df['ack'] = predictor_train
test_df = independants_test
test_df['ack'] = predictor_test

class OneRClassif:
    def __init__(self, max_depth=1) -> None:
        self.rules = dict()
        self.label_encoder = LabelEncoder()
        self.best_column = None
        self.lowest_error = 1
        self.max_depth = max_depth

    def fit(self, X):
        length_df = len(X)
        check_len = length_df
        teoretic_MAX = 0
        while check_len > 10:
            check_len /= 10
            teoretic_MAX += 1
        if self.max_depth == teoretic_MAX:
            print(f"cannot fit data because length of set doesnt allow depth of {self.max_depth} changing max depth to {teoretic_MAX - 1}")
            self.max_depth = teoretic_MAX - 1

        for column in X:
            if column == 'ack':
                continue
            df = X[[column, 'ack']]
            if pd.api.types.is_numeric_dtype(X[column]):  # is numeric column
                df = df.sort_values(by=column, ascending=True).reset_index()
                # rozdelenie na 10 rangeov
                part_size = length_df // 10
                ranges = {}
                total_error_rate = 0

                for i in range(10):
                    start_index = i * part_size
                    end_index = start_index + part_size if i < 9 else length_df

                    majority = self.find_majority(df[start_index: end_index])
                    if start_index == 0:
                        ranges[(np.finfo(np.float64).min, df[column][end_index])] = majority[0]
                    elif i == 9:
                        ranges[(df[column][start_index], np.finfo(np.float64).max )] = majority[0]
                    else:
                        ranges[(df[column][start_index], df[column][end_index])] = majority[0]
                    total_error_rate += majority[1] / length_df
                if total_error_rate < self.lowest_error:
                    self.lowest_error = total_error_rate
                    self.best_column = column
                    self.rules = ranges
            else:
                # df[column] = self.label_encoder.fit_transform(df[column]) nefunguje to aj tak
                pd.options.mode.chained_assignment = None
                df[column] = df[column].astype('category')
                unique_cats = df[column].cat.categories
                categories = {}
                total_error_rate = 0
                for category in unique_cats:
                    majority = self.find_majority(df[df[column] == category])
                    total_error_rate += majority[1] / length_df
                    categories[(category,)] = majority[0]
                if total_error_rate < self.lowest_error:
                    self.lowest_error = total_error_rate
                    self.best_column = column
                    self.rules = categories
        self.rules["name"] = self.best_column
        if self.max_depth > 0:
            # print(self.rules)
            for key in self.rules.keys():
                if key == "name":
                    continue
                if pd.api.types.is_numeric_dtype(X[self.best_column]):
                    new_OneR = OneRClassif(self.max_depth - 1)
                    new_OneR = new_OneR.fit(X[(key[0] <= X[self.best_column]) & (key[1] >= X[self.best_column])])
                    self.rules[key] = new_OneR.rules
                else:
                    new_OneR = OneRClassif(self.max_depth - 1)
                    new_OneR = new_OneR.fit(X[X[self.best_column] == key])
                    self.rules[key] = new_OneR.rules
        return self

    def find_majority(self, df):
        yes = df['ack'].sum()
        no = len(df) - yes
        if yes > no:
            return 1, no
        else:
            return 0, yes

    def predict(self, X):
        value = X[self.rules['name']]
        current_rules = self.rules
        while True:
            if pd.api.types.is_numeric_dtype(value):
                for key in current_rules:
                    if key[0] <= value <= key[1]:
                        current_rules = current_rules[key]
                        break

            else:
                for key in current_rules:
                    if key == value:
                        current_rules = current_rules[key]
                        break
            if current_rules in [0,1]:
                return current_rules





newOne = OneRClassif(max_depth=0)
newOne.fit(temp_df)
result = []
correct = 0
true_pos  = 0
true_negative = 0
false_pos = 0
false_negative = 0
for i in range(len(test_df)):
        result.append(newOne.predict(test_df.iloc[i]))
        if result[-1] == 1 and test_df["ack"].iloc[i] == 1:
            true_pos += 1
        elif result[-1] == 1 and test_df["ack"].iloc[i] == 0:
            false_pos += 1
        elif result[-1] == 0 and test_df["ack"].iloc[i] == 0:
            true_negative += 1
        elif result[-1] == 0 and test_df["ack"].iloc[i] == 1:
            false_negative += 1
result_frame = pd.DataFrame(result)
print(result_frame, test_df["ack"])
print("Accuracy: ",(true_pos + true_negative) / (true_pos + true_negative + false_pos + false_negative))
print("Precision: ", true_pos / (true_pos + false_pos))
print("Recall: ", true_pos / (true_pos + false_negative))


     0
0    1
1    0
2    1
3    1
4    0
..  ..
810  0
811  0
812  1
813  1
814  1

[815 rows x 1 columns] 0      0.0
1      0.0
2      1.0
3      1.0
4      0.0
      ... 
810    1.0
811    0.0
812    1.0
813    1.0
814    1.0
Name: ack, Length: 815, dtype: float64
Accuracy:  0.7325153374233129
Precision:  0.777511961722488
Recall:  0.7222222222222222
